In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [3]:
from torchvision import datasets, transforms
data_path = '/data-unversioned/p1ch6/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(size=[32,32], padding=4),                        
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))


Extracting /data-unversioned/p1ch6/cifar-10-python.tar.gz to /data-unversioned/p1ch6/


In [4]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [5]:
class resblock(nn.Module):
    def __init__(self, n):
        super(resblock, self).__init__()
        self.conv = nn.Conv2d(n, n, kernel_size=3,
                              padding=1, bias=False)  
        self.batch_norm = nn.BatchNorm2d(num_features=n)

        torch.nn.init.kaiming_normal_(self.conv.weight,
                                      nonlinearity='relu')  
        torch.nn.init.constant_(self.batch_norm.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm.bias)

    def forward(self, x):
        out = self.conv(x)
        out = self.batch_norm(out)
        out = torch.relu(out)
        return out + x

In [6]:
class deep(nn.Module):
    def __init__(self, n1=32, n_blocks=10):
        super().__init__()
        self.n1 = n1
        self.conv1 = nn.Conv2d(3, n1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n1, 2*n1, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(2*n1, 4*n1, kernel_size=3, padding=1)
        self.resblocks = nn.Sequential(
            *(n_blocks * [resblock(n=n1)]))
        self.resblocks2 = nn.Sequential(
            *(n_blocks * [resblock(n=n1*2)]))
        self.resblocks3 = nn.Sequential(
            *(n_blocks * [resblock(n=n1*4)]))
        self.fc1 = nn.Linear(8 * 8 * n1*4, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 10)
        
    def forward(self, x):
        out = torch.relu(self.conv1(x))
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2) 
        out = self.conv2(out)
        out = self.resblocks2(out)
        out = F.max_pool2d(out, 2) 
        out = self.conv3(out)
        out = self.resblocks3(out)
        out = out.view(-1, 8 * 8 * self.n1*4)
        out = torch.relu(self.fc1(out))
        out = torch.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [7]:
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [8]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)  # <1>
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        
        print('{} Epoch {}, Training loss {}'.format(
            datetime.datetime.now(), epoch,
            loss_train / len(train_loader)))

In [9]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=True)

model =  deep().to(device=device)# <1>
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

In [10]:
numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list

(550106,
 [864,
  32,
  18432,
  64,
  73728,
  128,
  9216,
  32,
  32,
  36864,
  64,
  64,
  147456,
  128,
  128,
  262144,
  32,
  512,
  16,
  160,
  10])

In [11]:
training_loop(  
    n_epochs = 120,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


2021-07-29 01:27:02.471632 Epoch 1, Training loss 1.6422754122168206
2021-07-29 01:27:59.010880 Epoch 2, Training loss 1.2976933411320153
2021-07-29 01:28:55.709591 Epoch 3, Training loss 1.111542597908498
2021-07-29 01:29:52.204132 Epoch 4, Training loss 0.9975588881146268
2021-07-29 01:30:48.819054 Epoch 5, Training loss 0.9062440858015319
2021-07-29 01:31:44.997942 Epoch 6, Training loss 0.8485212753648343
2021-07-29 01:32:41.886670 Epoch 7, Training loss 0.7981090415316774
2021-07-29 01:33:38.348687 Epoch 8, Training loss 0.7584562593561304
2021-07-29 01:34:34.891318 Epoch 9, Training loss 0.7279200221952575
2021-07-29 01:35:31.089570 Epoch 10, Training loss 0.6872501747916117
2021-07-29 01:36:27.384306 Epoch 11, Training loss 0.6666053393307854
2021-07-29 01:37:24.279359 Epoch 12, Training loss 0.6398356780600365
2021-07-29 01:38:20.608371 Epoch 13, Training loss 0.6074812681321293
2021-07-29 01:39:16.801910 Epoch 14, Training loss 0.5955316850062832
2021-07-29 01:40:13.177254 Epo

In [ ]:
torch.save(model.state_dict(), r'C:\Users\hp\Desktop\me\resnet20.pt')


In [12]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():  # <1>
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <2>
                total += labels.shape[0]  # <3>
                correct += int((predicted == labels).sum())  # <4>

        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model, train_loader, val_loader)

Accuracy train: 0.97
Accuracy val: 0.84
